# Tutorial 1 — Backend Prediction Comparison

This notebook demonstrates how to:
1. Create a simple U-Net model with skip connections
2. Convert it to HLS using both **VitisUnified** and **Vitis** backends
3. Compare prediction results between the two backends

Both `axis` (AXI-Stream) and `axim` (AXI-Master) modes are tested.

## 1. Imports & Environment Setup

In [ ]:
import os
from pathlib import Path

import numpy as np
from tensorflow.keras.layers import (
    Concatenate,
    Conv2D,
    Input,
    MaxPooling2D,
    UpSampling2D,
)
from tensorflow.keras.models import Model, load_model

import hls4ml
import hls4ml.model

In [ ]:
# ── Paths ──────────────────────────────────────────────
test_root_path = Path(".").resolve()

os.environ["XILINX_VITIS"] = "/tools/Xilinx/Vitis/2023.2"
os.environ["PATH"] = os.environ["XILINX_VITIS"] + "/bin:" + os.environ["PATH"]

# Platform file used by both axi_mode='axim' (vendor) and axi_mode='axis' (custom)
XPFM_PATH_CUSTOM = (
    "/tools/Xilinx/Vitis/2023.2/base_platforms/"
    "xilinx_zcu102_base_202320_1/xilinx_zcu102_base_202320_1.xpfm"
)
XPFM_PATH_VENDOR = XPFM_PATH_CUSTOM  # same for this example

## 2. Helper Functions

In [ ]:
def create_io_file_dir():
    os.makedirs(test_root_path / "input_file", exist_ok=True)
    os.makedirs(test_root_path / "output_file", exist_ok=True)


def checkEqual(a, b):
    equal = np.array_equal(a, b)
    if equal:
        print("Test pass — both are equal \U0001f642")
    else:
        print("Test fail — both are NOT equal \U0001f62c")
    return equal


def create_simple_testcase(inputShape=(4, 4, 1), fileName="inputX.npy"):
    n_in = np.random.rand(*inputShape).astype(np.float32)
    os.makedirs(test_root_path / "input_file", exist_ok=True)
    np.save(test_root_path / "input_file" / fileName, n_in)


def create_simple_unet(input_shape=(4, 4, 1), modelName="simpleSkip.keras"):
    inputs = Input(input_shape)
    # Encoder
    c1 = Conv2D(2, (3, 3), activation="relu", padding="same")(inputs)
    p1 = MaxPooling2D((2, 2))(c1)
    # Bottleneck
    bn = Conv2D(4, (3, 3), activation="relu", padding="same")(p1)
    # Decoder
    u1 = UpSampling2D((2, 2))(bn)
    concat1 = Concatenate()([u1, c1])
    c2 = Conv2D(2, (3, 3), activation="relu", padding="same")(concat1)
    # Output layer (1 channel)
    outputs = Conv2D(1, (1, 1), activation="sigmoid")(c2)
    model = Model(inputs, outputs)
    model.compile(optimizer="adam", loss="binary_crossentropy")
    model.save(test_root_path / "input_file" / modelName)
    return model


def gen_prj_dir(backend, io_type, strategy, granularity, prefix, axi_mode):
    return str(
        test_root_path
        / f"hls4mlprj_{prefix}_{backend}_{strategy}_{io_type}_{granularity}_{axi_mode}"
    )

In [ ]:
def create_hls_model(
    model, config, backend, io_type, strategy, granularity, prefix, axi_mode
):
    output_dir = gen_prj_dir(backend, io_type, strategy, granularity, prefix, axi_mode)
    hls_model = hls4ml.converters.convert_from_keras_model(
        model,
        hls_config=config,
        output_dir=output_dir,
        backend=backend,
        io_type=io_type,
        board="zcu102",
        part="xczu9eg-ffvb1156-2-e",
        clock_period="10ns",
        input_type="float",
        output_type="float",
        xpfmPath=XPFM_PATH_VENDOR if axi_mode == "axim" else XPFM_PATH_CUSTOM,
        axi_mode=axi_mode,
    )
    hls_model.compile()
    return hls_model

## 3. Parameters

In [ ]:
io_type     = "io_stream"
strategy    = "latency"
granularity = "name"
amt_query   = 10
axi_mode    = "axim"        # change to "axis" to test AXI-Stream mode

## 4. Prepare Data & Model

In [ ]:
create_io_file_dir()

# Create and load test data
create_simple_testcase(inputShape=(amt_query, 4, 4, 1), fileName="inputX.npy")
input_data = np.load(test_root_path / "input_file" / "inputX.npy")
print(f"Input shape: {input_data.shape}")

# Create and load model
model_name = "simpleSkip.keras"
create_simple_unet(modelName=model_name)
model = load_model(test_root_path / "input_file" / model_name)
model.summary()

## 5. Configure & Convert

In [ ]:
config = hls4ml.utils.config_from_keras_model(model, granularity=granularity)
config

In [ ]:
# Convert with VitisUnified backend
vitis_unified_model = create_hls_model(
    model, config, "VitisUnified", io_type, strategy, granularity, "bridge", axi_mode
)
print("VitisUnified model compiled successfully.")

In [ ]:
# Convert with Vitis backend
vitis_model = create_hls_model(
    model, config, "Vitis", io_type, strategy, granularity, "bridge", axi_mode
)
print("Vitis model compiled successfully.")

## 6. Predict & Compare

In [ ]:
y_hls4ml_unified = vitis_unified_model.predict(input_data)
y_hls4ml         = vitis_model.predict(input_data)

print(f"VitisUnified output shape: {np.array(y_hls4ml_unified).shape}")
print(f"Vitis        output shape: {np.array(y_hls4ml).shape}")

assert checkEqual(y_hls4ml_unified, y_hls4ml), (
    "The results from VitisUnified and Vitis are NOT equal!"
)
print("\n✅ Backend prediction comparison passed.")